In [3]:
import torch
assert torch.cuda.is_available()

In [2]:
batch_size = 4

In [ ]:
# from datasets import load_dataset

# # Login using e.g. `huggingface-cli login` to access this dataset
# ds = load_dataset("yale-nlp/FOLIO")

# train_df = ds["train"].to_pandas()
# validation_df = ds["validation"].to_pandas()

/uufs/chpc.utah.edu/common/home/u1283221/miniconda3/envs/linc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
import pandas as pd
train_df = pd.read_json("FOLIO/data/v0.0/folio-train.jsonl", lines=True)
validation_df = pd.read_json("FOLIO/data/v0.0/folio-validation.jsonl", lines=True)

In [19]:
train_df.iloc[125]

story_id                                                      481
example_id                                                   1408
conclusion                        A worksheet is not dispensable.
premises        [All dispensable things are environment-friend...
premises-FOL    [∀x (Dispensable(x) → EnvironmentFriendly(x)),...
label                                                     Unknown
source                                                        hyb
Name: 125, dtype: object

In [ ]:
# train_df["premises"] = train_df["premises"].apply(lambda x: "\n".join(x))
# train_df["premises-FOL "] = train_df["premises-FOL "].apply(lambda x: "\n".join(x))

In [20]:
in_context_indices = [126, 24,61, 276, 149, 262, 264, 684]
in_context_examples = train_df.iloc[[x - 1 for x in in_context_indices]]

In [21]:
in_context_format = """\
<PREMISES>
{premises}
</PREMISES>
<CONCLUSION>
{conclusion}
</CONCLUSION>
<EVALUATE>
{premises_FOL}
{label}
</EVALUATE>\
"""

In [57]:
for index, in_context_example in in_context_examples.iterrows():
        print(in_context_format.format(premises = "\n".join(in_context_example["premises"]), conclusion = in_context_example["conclusion"], premises_FOL = "\n".join(in_context_example["premises-FOL"]), label = in_context_example["label"]))
        print()

<PREMISES>
All dispensable things are environment-friendly. 
All woodware is dispensable.
All paper is woodware. 
No good things are bad. 
All environment-friendly things are good.
A worksheet is either paper or is environment-friendly.
</PREMISES>
<CONCLUSION>
A worksheet is not dispensable.
</CONCLUSION>
<EVALUATE>
∀x (Dispensable(x) → EnvironmentFriendly(x))
∀x (Woodware(x) → Dispensable(x))
∀x (Paper(x) → Woodware(x))
∀x (Good(x) → ¬Bad(x))
∀x (EnvironmentFriendly(x) → Good(x))
Paper(worksheet) ⊕ EnvironmentFriendly(worksheet)
Unknown
</EVALUATE>

<PREMISES>
A La Liga soccer team ranks higher than another if it receives more points.
If two La Liga soccer teams recieve the same points, the team which recieves more points from the games between the two teams ranks higher.
Real Madrid and Barcelona are both La Liga soccer teams.
In La Liga 2021-2022, Real Madrid recieves 86 points and Barcelon recieves 73 points.
In La Liga 2021-2022, Real Madrid and Barcelona both recieve 3 points fr

In [ ]:
error_indices = [3, 109, 110, 111, 6, 28, 30, 48, 113, 115, 139, 140, 10, 11, 12, 88, 106, 107, 108, 174, 175, 176]
validation_df = validation_df.drop([x - 1 for x in error_indices])

In [5]:
validation_df

,story_id,premises,premises-FOL,conclusion,conclusion-FOL,label,example_id
0,380,People in this club who perform in school tale...,"∀x (InThisClub(x) ∧ PerformOftenIn(x, schoolTa...",Bonnie performs in school talent shows often.,"InThisClub(bonnie) ∧ (Perform(bonnie, schoolTa...",Uncertain,1014
1,380,People in this club who perform in school tale...,"∀x (InThisClub(x) ∧ PerformOftenIn(x, schoolTa...",If Bonnie is either both a young child or teen...,¬((YoungChildren(bonnie) ⊕ Teenager(bonnie)) ∧...,True,1015
2,380,People in this club who perform in school tale...,"∀x (InThisClub(x) ∧ PerformOftenIn(x, schoolTa...",If Bonnie either chaperones high school dances...,"(Chaperone(bonnie, highSchoolDance) ⊕ Perform(...",False,1016
4,426,All employees who schedule a meeting with thei...,"∀x ((Employee(x) ∧ Schedule(x, meeting, custom...",James does not have lunch in the company.,"¬HasLunch(james, company)",Uncertain,1209
5,426,All employees who schedule a meeting with thei...,"∀x ((Employee(x) ∧ Schedule(x, meeting, custom...",If James is either a manager or in other count...,"Manager(james) → ¬Work(james, home)",True,1210
...,...,...,...,...,...,...,...
198,101,"Ailton Silva, born in 1995, is commonly known ...","BornIn(ailtonSilva, year1995) ∧ CommonlyKnownA...",No one playing for Nautico is Brazilian.,"∀x (PlayFor(x, nautico) → ¬Brazilian(x))",False,304
199,101,"Ailton Silva, born in 1995, is commonly known ...","BornIn(ailtonSilva, year1995) ∧ CommonlyKnownA...",Ailton Silva does not play for a football club.,"∀x (FootballClub(x) → ¬PlayFor(ailtonSilva, x))",False,305
200,101,"Ailton Silva, born in 1995, is commonly known ...","BornIn(ailtonSilva, year1995) ∧ CommonlyKnownA...",Ailton was not loaned out to a football club.,"∀x (FootballClub(x) → ¬LoanedTo(ailton, x))",False,306
201,101,"Ailton Silva, born in 1995, is commonly known ...","BornIn(ailtonSilva, year1995) ∧ CommonlyKnownA...",Ailton Silva played for Fluminense.,"PlayFor(ailtonSilva, fluminense)",Uncertain,307


In [6]:
folio_prompt = """\
The following is a first-order logic (FOL) problem.
The problem is to determine whether the conclusion follows from the premises.
The premises are given in the form of a set of first-order logic sentences.
The conclusion is given in the form of a single first-order logic sentence.
The task is to translate each of the premises and conclusions into FOL expressions, so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.
Expressions should be adhere to the format of the Python NLTK package logic module:

The following is an example of a problem:
<PREMISES>
All dispensable things are environment-friendly.
All woodware is dispensable.
All paper is woodware.
No good things are bad.
All environment-friendly things are good.
A worksheet is either paper or is environment-friendly.
</PREMISES>
<CONCLUSION>
A worksheet is not dispensable.
</CONCLUSION>

The problem is answered in this format:
<EVALUATE>
TEXT: All dispensable things are environment-friendly.
FOL: all x. (Dispensable(x) -> EnvironmentFriendly(x))
TEXT: All woodware is dispensable.
FOL: all x. (Woodware(x) -> Dispensable(x))
TEXT: All paper is woodware.
FOL: all x. (Paper(x) -> Woodware(x))
TEXT: No good things are bad.
FOL: all x. (Good(x) -> -Bad(x))
TEXT: All environment-friendly things are good.
FOL: all x. (EnvironmentFriendly(x) -> Good(x))
TEXT: A worksheet is either paper or is environment-friendly.
FOL: ((Paper(Worksheet) & -EnvironmentFriendly( Worksheet)) | (-Paper(Worksheet) & EnvironmentFriendly(Worksheet)))
TEXT: A worksheet is not dispensable.
FOL: -Dispensable(Worksheet)
</EVALUATE>

<PREMISES>
{premises}
</PREMISES>
<CONCLUSION>
{conclusion}
</CONCLUSION>

<EVALUATE>\
"""

def format_prompt(premises, conclusion):
    return folio_prompt.format(premises=premises, conclusion=conclusion)

example_row = validation_df.iloc[0]
print(format_prompt(example_row.premises, example_row.conclusion))

The following is a first-order logic (FOL) problem.
The problem is to determine whether the conclusion follows from the premises.
The premises are given in the form of a set of first-order logic sentences.
The conclusion is given in the form of a single first-order logic sentence.
The task is to translate each of the premises and conclusions into FOL expressions, so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.
Expressions should be adhere to the format of the Python NLTK package logic module:

The following is an example of a problem:
<PREMISES>
All dispensable things are environment-friendly.
All woodware is dispensable.
All paper is woodware.
No good things are bad.
All environment-friendly things are good.
A worksheet is either paper or is environment-friendly.
</PREMISES>
<CONCLUSION>
A worksheet is not dispensable.
</CONCLUSION>

The problem is answered in this format:
<EVALUATE>
TEXT: All dispensable thin

In [35]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# model = "bigcode/starcoder2-3b"
model = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline("text-generation", model=model, device_map="auto", torch_dtype=torch.float16)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.09s/it]


In [ ]:
prompt = format_prompt(example_row.premises, example_row.conclusion)
chat = [
    {"role": "user", "content": prompt}
]
response = pipe(chat, max_new_tokens=256)[0]['generated_text'][-1]['content']

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [37]:
print(example_row.premises)

People in this club who perform in school talent shows often attend and are very engaged with school events.
People in this club either perform in school talent shows often or are inactive and disinterested community members.
People in this club who chaperone high school dances are not students who attend the school.
All people in this club who are inactive and disinterested members of their community chaperone high school dances.
All young children and teenagers in this club who wish to further their academic careers and educational opportunities are students who attend the school. 
Bonnie is in this club and she either both attends and is very engaged with school events and is a student who attends the school or is not someone who both attends and is very engaged with school events and is not a student who attends the school.


In [38]:
print(example_row.conclusion)

Bonnie performs in school talent shows often.


In [39]:
print(response)

Here's a Python function that translates the given FOL problem into a format that can be evaluated by a theorem solver.

```python
def translate_fol_problem(prems, concl):
    # Define the predicates
    predicates = {
        'Dispensable': 'Dispensable',
        'EnvironmentFriendly': 'EnvironmentFriendly',
        'Woodware': 'Woodware',
        'Paper': 'Paper',
        'Good': 'Good',
        'Bad': 'Bad',
        'Worksheet': 'Worksheet'
    }

    # Define the logical operators
    operators = {
        'all': 'all',
        'not': 'not',
        'and': '&',
        'or': '|',
        'implies': '->',
        'equivalent': '<->'
    }

    # Translate the premises
    translated_prems = []
    for prems_str in prems:
        if 'All' in prems_str:
            if 'and' in prems_str:
                if 'or' in prems_str:
                    translated_prems.append(f'{operators["all"]} x. ({prems_str.split("and")[0].strip()} & {prems_str.split("or")[1].strip()})')
                e

In [1]:
from nltk.sem.logic import LogicParser
from nltk.inference.prover9 import Prover9

def extract_fol_strings(text):
    lines = text.split("\n")
    fol_lines = [line[5:] for line in lines if "FOL:" in line]
    return fol_lines

def convert_fol_exps(fol_strings):
    tlp = LogicParser(True)
    fol_exps = [tlp.parse(fol_string) for fol_string in fol_strings]
    return fol_exps

prover = Prover9()

In [2]:
import os
os.environ['PROVER9'] = '/uufs/chpc.utah.edu/common/home/u1283221/cs6964/linc/prover9/source/bin'
print(os.environ['PROVER9'])

/uufs/chpc.utah.edu/common/home/u1283221/cs6964/linc/prover9/source/bin


In [3]:
answered_prompt_example = """\
<EVALUATE>
TEXT: All athletes are good at sports.
FOL: all x. (Athlete(x) -> GoodAtSports(x))
TEXT: All Olympic gold medal winners are good athletes.
FOL: all x. (OlympicGoldMedalWinner(x) -> Athlete(x))
TEXT: No scientists are good at sports.
FOL: all x. (Scientist(x) -> -GoodAtSports(x))
TEXT: All Nobel laureates are scientists.
FOL: all x. (NobelLaureate(x) -> Scientist(x))
TEXT: Amy is good at sports or Amy is an Olympic gold medal winner.
FOL: GoodAtSports(amy) | OlympicGoldMedalWinner(amy)
TEXT: If Amy is not a Nobel laureate, then Amy is not an Olympic gold medal winner.
FOL: -NobelLaureate(amy) -> -OlympicGoldMedalWinner(amy)
TEXT: If Amy is not an Olympic gold medal winner, then Amy is a Nobel laureate.
FOL: -OlympicGoldMedalWinner(amy) -> NobelLaureate(amy)
</EVALUATE>\
"""

example_fol_strings = extract_fol_strings(answered_prompt_example)
example_fol_exps = convert_fol_exps(example_fol_strings)
prover.prove(example_fol_exps[-1], example_fol_exps[:-1])

False

In [13]:
tlp = LogicParser(True)
tlp.parse("all x all y (LaLiga(x) & LaLiga(y) & MorePoints(x, y) -> HigherRank(x, y))") 

<AllExpression all x y.((LaLiga(x) & LaLiga(y) & MorePoints(x,y)) -> HigherRank(x,y))>

In [29]:
example_fol_strings = extract_fol_strings(response)
example_fol_exps = convert_fol_exps(example_fol_strings)
prover.prove(example_fol_exps[-1], example_fol_exps[:-1])

LogicalExpressionException: Unexpected token: '&'.  Expected token ')'.
Bonnie(ClubMember(Bonnie) & ((Attends(Bonnie) & VeryEngaged(Bonnie) & Attends(Bonnie)) | (-Attends(Bonnie) & -VeryEngaged(Bonnie) & -Attends(Bonnie))))
                          ^

In [15]:
example_row["premises-FOL"].split("\n")

['∀x (InThisClub(x) ∧ PerformOftenIn(x, schoolTalentShow) → Attend(x, schoolEvent) ∧ VeryEngagedWith(x, schoolEvent))',
 '∀x (InThisClub(x) → PerformOftenIn(x, schoolTalentShow) ⊕ (InActive(x) ∧ Disinterested(x) ∧ MemberOf(x, community)))',
 '∀x (InThisClub(x) ∧ Chaperone(x, highSchoolDance) → ¬(Studen(x) ∧ AttendSchool(x)))',
 '∀x (InThisClub(x) ∧ (InActive(x) ∧ Disinterested(x) ∧ MemberOf(x, community)) → Chaperone(x, highSchoolDances))',
 '∀x (InThisClub(x) ∧ (YoungChildren(x) ⊕ Teenager(x)) ∧ WishToFurther(x, academicCareer)) → Studen(x) ∧ AttendSchool(x))',
 'InThisClub(bonnie) ∧ ¬((Attend(x, schoolEvent) ∧ VeryEngagedWith(bonnie, schoolEvent)) ⊕ (Studen(bonne) ∧ AttendSchool(bonnie)))']

In [ ]:
validation_df.iloc[3]["premises-FOL"]

'∀x ((Employee(x) ∧ Schedule(x, meeting, customers)) → AppearIn(x, company))\n∀x ((Employee(x) ∧ HasLunch(x, company)) → Schedule(x, meeting, customers))\n∀x (Employee(x) → (HasLunch(x, company) ⊕ HasLunch(x, home)))\n∀x ((Employee(x) ∧ HasLunch(x, home)) → Work(x, home))\n∀x ((Employee(x) ∧ (¬In(x, homecountry))) → Work(x, home))\n∀x (Manager(x) → ¬Work(x, home))\n¬(Manager(james) ⊕ AppearIn(james, company))'

In [ ]:
# folio_symbols_map = {
#     "¬": "-",
#     "∧": "&",
#     "∨": "|",
#     "∀": "all ",
# }

In [16]:
convert_fol_exps(example_row["premises-FOL"].split("\n"))

# prover.prove(convert_fol_exps(, ))

LogicalExpressionException: Unexpected token: '∧'.  Expected token ')'.
∀x (InThisClub(x) ∧ PerformOftenIn(x, schoolTalentShow) → Attend(x, schoolEvent) ∧ VeryEngagedWith(x, schoolEvent))
                  ^